In [1]:
!pip install lxml

In [3]:
import pandas as pd
import glob
import os
import re

# --- Configuration ---
# Set the path to the folder containing your .xls files
PATH = "Area Under Crops/"

# IMPORTANT: Inspect one of your Excel files to find the correct row numbers.
# Row numbers start from 0.
# - SKIP_ROWS: The number of rows to skip at the very top of the file.
# - HEADER_ROWS: A list containing the row numbers of your column titles.
#   Based on your image, the headers are likely on the 4th and 5th rows.
SKIP_ROWS = 3
HEADER_ROWS = [0, 1]
# -----------------------------------------------------------

all_files = glob.glob(os.path.join(PATH, "*.xls"))
print(f"Found {len(all_files)} files to process.")

all_dataframes = []

for file in all_files:
    print(f"Processing {os.path.basename(file)}...")
    try:
        # 1. Read the file, specifying where the multi-level header is located.
        # We use a try-except block to handle files that might be HTML.
        try:
            df = pd.read_excel(file, header=HEADER_ROWS, skiprows=SKIP_ROWS, engine="xlrd")
        except Exception:
            df = pd.read_html(file, header=HEADER_ROWS, skiprows=SKIP_ROWS)[0]

        # 2. Flatten the multi-level column headers into a single level.
        # e.g., ('Rice', 'Kharif') becomes 'Rice_Kharif'
        flat_columns = []
        for col_level1, col_level2 in df.columns:
            col_level1 = "" if "unnamed" in str(col_level1).lower() else str(col_level1).strip()
            col_level2 = "" if "unnamed" in str(col_level2).lower() else str(col_level2).strip()
            
            # Combine levels, handling cases where one level is empty
            if col_level1 and col_level2:
                flat_columns.append(f"{col_level1}_{col_level2}")
            else:
                flat_columns.append(col_level1 or col_level2)
        
        df.columns = flat_columns
        
        # Rename the first two columns consistently for easier processing
        df.rename(columns={df.columns[0]: 'S_No', df.columns[1]: 'District'}, inplace=True)

        # 3. Create a 'State' column and propagate the state name to all its district rows.
        # State rows are identified as those where 'S_No' is empty but 'District' has a name.
        df['State'] = df.where(df['S_No'].isna() & df['District'].notna())['District']
        df['State'].ffill(inplace=True) # Forward-fill the state name

        # 4. Clean the DataFrame
        # Remove original state-header rows and any rows that are 'Total'
        df.dropna(subset=['S_No'], inplace=True) # Drops rows without a serial number
        df = df[~df['District'].str.contains('total', case=False, na=False)].copy()

        # Add a 'Year' column from the filename
        match = re.search(r"(\d{4})", os.path.basename(file))
        if match:
            df['Year'] = int(match.group(1))

        all_dataframes.append(df)

    except Exception as e:
        print(f"Could not process {os.path.basename(file)}: {e}")

# --- Merge and Finalize ---
if all_dataframes:
    # Concatenate all the processed DataFrames into one
    final_df = pd.concat(all_dataframes, ignore_index=True)
    
    # Reorder columns for better readability and remove the original 'S_No'
    cols_to_move = ['Year', 'State', 'District']
    final_df = final_df[cols_to_move + [col for col in final_df.columns if col not in cols_to_move]]
    final_df = final_df.drop(columns=['S_No'], errors='ignore')
    
    print("\n Successfully merged all files!")
    print(f"Final DataFrame has {final_df.shape[0]} rows and {final_df.shape[1]} columns.")
    
    # Save the clean data to a new CSV file
    final_df.to_csv("merged_crop_data_clean.csv", index=False)
    print("\n Data saved to 'merged_crop_data_clean.csv'")

    # To display the first 5 rows in Jupyter, run `final_df.head()` in a new cell
    display(final_df.head())
else:
    print("\n No files were processed. The final DataFrame is empty.")

Found 0 files to process.

 No files were processed. The final DataFrame is empty.


In [4]:
final_df.to_csv('Area_under_crops.csv')

NameError: name 'final_df' is not defined

In [3]:
import pandas as pd
import glob
import os
import re

# --- Configuration ---
PATH = "Crop Irrigated Area/"
SKIP_ROWS = 2
HEADER_ROWS = [0, 1]
# -----------------------------------------------------------

all_files = glob.glob(os.path.join(PATH, "*.xls"))
print(f"Found {len(all_files)} files to process.")

all_dataframes = []

for file in all_files:
    print(f"Processing {os.path.basename(file)}...")
    try:
        # Step 1: Read the file
        try:
            df = pd.read_excel(file, header=HEADER_ROWS, skiprows=SKIP_ROWS, engine="xlrd")
        except Exception:
            df = pd.read_html(file, header=HEADER_ROWS, skiprows=SKIP_ROWS)[0]

        # --- FIXED SECTION: MORE ROBUST COLUMN FLATTENING ---
        flat_columns = []
        # Using enumerate to get the column index `i` for unique fallback names
        for i, col_tuple in enumerate(df.columns):
            level1 = str(col_tuple[0])
            level2 = str(col_tuple[1])

            # Clean each level by removing pandas' default 'Unnamed' text
            clean_l1 = "" if 'unnamed' in level1.lower() else level1.strip()
            clean_l2 = "" if 'unnamed' in level2.lower() else level2.strip()

            # Combine the cleaned levels into a final name
            if clean_l1 and clean_l2:
                final_name = f"{clean_l1}_{clean_l2}"
            # Use the level that isn't empty
            elif clean_l1:
                final_name = clean_l1
            elif clean_l2:
                final_name = clean_l2
            # If both header levels were blank, create a unique fallback name
            else:
                final_name = f"unnamed_col_{i}"
            
            flat_columns.append(final_name)
        
        # This is a final safety check to ensure 100% uniqueness.
        # If any names are still duplicated, it adds a suffix (_2, _3, etc.)
        counts = {}
        unique_columns = []
        for col in flat_columns:
            if col in counts:
                counts[col] += 1
                unique_columns.append(f"{col}_{counts[col]}")
            else:
                counts[col] = 1 # Start count at 1
                unique_columns.append(col)
        
        df.columns = unique_columns
        # --- END OF FIXED SECTION ---
        
        # Rename the first two columns for easy access
        df.rename(columns={df.columns[0]: 'S_No', df.columns[1]: 'District'}, inplace=True)

        # Step 3: Create and forward-fill the 'State' column.
        df['State'] = df.where(df['S_No'].isna() & df['District'].notna())['District']
        df['State'].ffill(inplace=True)

        # Step 4: Clean the data.
        df.dropna(subset=['S_No'], inplace=True)
        df = df[~df['District'].str.contains('total', case=False, na=False)].copy()

        # Step 5: Add the 'Year' from the filename.
        match = re.search(r"(\d{4})", os.path.basename(file))
        if match:
            df['Year'] = int(match.group(1))

        all_dataframes.append(df)

    except Exception as e:
        print(f" Could not process {os.path.basename(file)}: {e}")

# --- Merge and Finalize ---
if all_dataframes:
    
    final_df = pd.concat(all_dataframes, ignore_index=True)
    
    cols_to_move = ['Year', 'State', 'District']
    final_df = final_df[cols_to_move + [col for col in final_df.columns if col not in cols_to_move]]
    final_df = final_df.drop(columns=['S_No'], errors='ignore')
    
    print("\n Successfully merged all files!")
    print(f"Final DataFrame has {final_df.shape[0]} rows and {final_df.shape[1]} columns.")
    
    final_df.to_csv("merged_crop_data_final.csv", index=False)
    print("\n Data saved to 'merged_crop_data_final.csv'")

    display(final_df.head())
else:
    print("\n No files were processed.")

Found 23 files to process.
Processing crop_irrigated_area_report2005.xls...
Processing crop_irrigated_area_report2011.xls...


/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/3712947425.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['State'].ffill(inplace=True)
/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/3712947425.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['State'].ffill(inpl

Processing crop_irrigated_area_report2010.xls...
Processing crop_irrigated_area_report2004.xls...


/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/3712947425.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['State'].ffill(inplace=True)
/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/3712947425.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['State'].ffill(inpl

Processing crop_irrigated_area_report2012.xls...
Processing crop_irrigated_area_report2006.xls...


...

/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/3712947425.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['State'].ffill(inplace=True)
/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/3712947425.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['State'].ffill(inpl

Processing crop_irrigated_area_report2008.xls...


/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/3712947425.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['State'].ffill(inplace=True)
/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/3712947425.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['State'].ffill(inpl


✅ Successfully merged all files!
Final DataFrame has 13235 rows and 64 columns.

💾 Data saved to 'merged_crop_data_final.csv'


,Year,State,District,Cereals & Millets_Rice,Cereals & Millets_Rice.1,Cereals & Millets_Rice.2,Cereals & Millets_Rice.3,Cereals & Millets_Rice.4,Cereals & Millets_Rice.5,Cereals & Millets_Jowar,...,"Drugs, Narcotics and plantation Crop_Coffee","Drugs, Narcotics and plantation Crop_Total Drugs, Narcotics and plantation Crop",Fodder Crops_Fodder Crops,Other Non Food Crops_Other Non Food Crops,Total Non Food Crop_Total Non Food Crop,Total Irrigated Area_Total Irrigated Area,unnamed_col_59,unnamed_col_60,Cereals & Millets_Jowar.3,unnamed_col_58
0,2005,NaN,District,Kharif,Rabi,Autumn,Winter,Summer,Total (3 To 7),Kharif,...,NaN,(47 + 48 + 49 + 50 + 51 + 52 + 53 + 54),NaN,NaN,(40 + 41 + 42 + 43 + 44 + 45 + 46 + 47 + 48 + ...,(32 + 58),NaN,NaN,NaN,NaN
1,2005,NaN,2,3,4,5,6,7,8,9,...,54,55,56.0,57.0,58,59,NaN,NaN,NaN,NaN
2,2005,NaN,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,NaN,Adilabad,NaN,NaN,53613,NaN,9438,63051,NaN,...,NaN,72,23.0,3.0,13169,109682,NaN,NaN,NaN,NaN
4,2005,NaN,Anantapur,NaN,NaN,24915,NaN,23142,48057,730,...,NaN,1,64.0,6091.0,58098,153606,NaN,NaN,NaN,NaN


In [4]:
final_df.to_csv('Crop_Irrigated_Area.csv')

In [19]:
import pandas as pd
import glob
import os
import re

# --- Configuration: Update these values! ---
PATH = "Source Irrigated Area/" 


# This format has a 3-level header.
SKIP_ROWS = 1 
HEADER_ROWS = [0, 1, 2] # This must be a list of THREE row numbers
# -----------------------------------------------------------

all_files = glob.glob(os.path.join(PATH, "*.xls"))
print(f"Found {len(all_files)} files to process.")

all_dataframes = []

for file in all_files:
    print(f"Processing {os.path.basename(file)}...")
    try:
        # Step 1: Read the file, specifying the 3-level header.
        try:
            df = pd.read_excel(file, header=HEADER_ROWS, skiprows=SKIP_ROWS, engine="xlrd")
        except Exception:
            df = pd.read_html(file, header=HEADER_ROWS, skiprows=SKIP_ROWS)[0]

        # --- UPDATED FOR 3-LEVEL HEADERS ---
        # This logic now correctly flattens the 3-level column names.
        # e.g., ('Net Irrigated Area', 'Well', 'Tubewell / Other Well') 
        # becomes 'Net Irrigated Area_Well_Tubewell / Other Well'
        flat_columns = []
        for i, col_tuple in enumerate(df.columns):
            # Clean each part of the 3-level tuple
            parts = [str(part) for part in col_tuple if 'unnamed' not in str(part).lower()]
            
            if parts:
                final_name = '_'.join(parts).strip()
            else:
                # Fallback for any completely blank columns
                final_name = f"unnamed_col_{i}"
            flat_columns.append(final_name)
        
        # Enforcing 100% uniqueness as a final safety measure
        counts = {}
        unique_columns = []
        for col in flat_columns:
            if col in counts:
                counts[col] += 1
                unique_columns.append(f"{col}_{counts[col]}")
            else:
                counts[col] = 1
                unique_columns.append(col)
        
        df.columns = unique_columns
        # --- END OF UPDATED SECTION ---
        
        # Rename the first two columns for easy access
        df.rename(columns={df.columns[0]: 'S_No', df.columns[1]: 'District'}, inplace=True)

        # Step 3: Create and forward-fill the 'State' column.
        df['State'] = df.where(df['S_No'].isna() & df['District'].notna())['District']
        df['State'].ffill(inplace=True)

        # Step 4: Clean the data.
        df.dropna(subset=['S_No'], inplace=True)
        df = df[~df['District'].str.contains('total', case=False, na=False)].copy()

        # Step 5: Add the 'Year' from the filename, if present.
        match = re.search(r"(\d{4})", os.path.basename(file))
        if match:
            df['Year'] = int(match.group(1))

        all_dataframes.append(df)

    except Exception as e:
        print(f" Could not process {os.path.basename(file)}: {e}")

# --- Merge and Finalize ---
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)
    
    # Reorder columns
    cols_to_move = ['Year', 'State', 'District']
    # Handle case where 'Year' might not be in all filenames
    cols_to_move = [col for col in cols_to_move if col in final_df.columns]
    
    final_df = final_df[cols_to_move + [col for col in final_df.columns if col not in cols_to_move]]
    final_df = final_df.drop(columns=['S_No'], errors='ignore')
    
    print("\n Successfully merged all files!")
    print(f"Final DataFrame has {final_df.shape[0]} rows and {final_df.shape[1]} columns.")
    
    final_df.to_csv("merged_irrigated_area_final.csv", index=False)
    print("\n Data saved to 'merged_irrigated_area_final.csv'")

    display(final_df.head())
else:
    print("\n No files were processed.")

Found 23 files to process.
Processing source_irrigated_area_report2018.xls...
Processing source_irrigated_area_report2019.xls...


/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/320780584.py:64: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['State'].ffill(inplace=True)
/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/320780584.py:64: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['State'].ffill(inplac

Processing source_irrigated_area_report2009.xls...
Processing source_irrigated_area_report2008.xls...
Processing source_irrigated_area_report2020.xls...


/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/320780584.py:64: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['State'].ffill(inplace=True)
/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/320780584.py:64: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['State'].ffill(inplac

Processing source_irrigated_area_report.2007xls.xls...
Processing source_irrigated_area_report1998.xls...
Processing source_irrigated_area_report1999.xls...


...

Processing source_irrigated_area_report2001.xls...
Processing source_irrigated_area_report2015.xls...
Processing source_irrigated_area_report2003.xls...


/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/320780584.py:64: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['State'].ffill(inplace=True)
/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/320780584.py:64: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['State'].ffill(inplac

Processing source_irrigated_area_report2017.xls...
Processing source_irrigated_area_report2016.xls...
Processing source_irrigated_area_report2002.xls...

✅ Successfully merged all files!
Final DataFrame has 12935 rows and 21 columns.

💾 Data saved to 'merged_irrigated_area_final.csv'


/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/320780584.py:64: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['State'].ffill(inplace=True)
/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/320780584.py:64: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['State'].ffill(inplac

,Year,State,District,Net Irrigated Area_Canal_Government,Net Irrigated Area_Canal_Private,Net Irrigated Area_Canal_Total (3 To 4),Net Irrigated Area_Tank_Tank,Net Irrigated Area_Well_Tubewell,Net Irrigated Area_Well_Other Well,Net Irrigated Area_Well_Total (6 To 7),...,Net Irrigated Area_Total (5 + 6 + 9 + 10)_Total (5 + 6 + 9 + 10),Gross Irrigated Area_Canal_Government,Gross Irrigated Area_Canal_Private,Gross Irrigated Area_Canal_Total (9 To 10),Gross Irrigated Area_Tank_Tank,Gross Irrigated Area_Well_Tubewell,Gross Irrigated Area_Well_Other Well,Gross Irrigated Area_Well_Total (12 To 13),Gross Irrigated Area_Other Source_Other Source,Gross Irrigated Area_Total (14 + 15 + 18 + 19)_Total (14 + 15 + 18 + 19)
0,2018,NaN,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
1,2018,NaN,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,...,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh
2,2018,NaN,Anantapur,14323,-,14323,1755,125330,195,125525,...,141991,16196,-,16196,2016,153088,195,153283,479,171974
3,2018,NaN,Chittoor,619,-,619,7496,124134,8261,132395,...,140510,619,-,619,7804,156601,10332,166933,-,175356
4,2018,NaN,East godavari,169922,-,169922,16793,68760,32,68792,...,277346,316411,-,316411,21248,115255,36,115291,27024,479974


In [20]:
final_df

,Year,State,District,Net Irrigated Area_Canal_Government,Net Irrigated Area_Canal_Private,Net Irrigated Area_Canal_Total (3 To 4),Net Irrigated Area_Tank_Tank,Net Irrigated Area_Well_Tubewell,Net Irrigated Area_Well_Other Well,Net Irrigated Area_Well_Total (6 To 7),...,Net Irrigated Area_Total (5 + 6 + 9 + 10)_Total (5 + 6 + 9 + 10),Gross Irrigated Area_Canal_Government,Gross Irrigated Area_Canal_Private,Gross Irrigated Area_Canal_Total (9 To 10),Gross Irrigated Area_Tank_Tank,Gross Irrigated Area_Well_Tubewell,Gross Irrigated Area_Well_Other Well,Gross Irrigated Area_Well_Total (12 To 13),Gross Irrigated Area_Other Source_Other Source,Gross Irrigated Area_Total (14 + 15 + 18 + 19)_Total (14 + 15 + 18 + 19)
0,2018,NaN,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
1,2018,NaN,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,...,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh,State: Andhra Pradesh
2,2018,NaN,Anantapur,14323,-,14323,1755,125330,195,125525,...,141991,16196,-,16196,2016,153088,195,153283,479,171974
3,2018,NaN,Chittoor,619,-,619,7496,124134,8261,132395,...,140510,619,-,619,7804,156601,10332,166933,-,175356
4,2018,NaN,East godavari,169922,-,169922,16793,68760,32,68792,...,277346,316411,-,316411,21248,115255,36,115291,27024,479974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12930,2002,NaN,Pithoragarh,1183,411,1594,-,-,-,-,...,4073,2155,584,2739,-,-,-,-,4970,7709
12931,2002,NaN,Rudra prayag,1079,147,1226,-,-,-,-,...,2363,2062,270,2332,-,-,-,-,2189,4521
12932,2002,NaN,Tehri garhwal,682,1324,2006,-,-,-,-,...,7913,1268,2532,3800,-,-,-,-,11258,15058
12933,2002,NaN,Udam singh nagar,26182,-,26182,42,99205,12338,111543,...,145226,56003,-,56003,1011,118922,58996,177918,1683,236615


In [21]:
final_df.columns

Index(['Year', 'State', 'District', 'Net Irrigated Area_Canal_Government',
       'Net Irrigated Area_Canal_Private',
       'Net Irrigated Area_Canal_Total (3 To 4)',
       'Net Irrigated Area_Tank_Tank', 'Net Irrigated Area_Well_Tubewell',
       'Net Irrigated Area_Well_Other Well',
       'Net Irrigated Area_Well_Total (6 To 7)',
       'Net Irrigated Area_Other Source_Other Source',
       'Net Irrigated Area_Total (5 + 6 + 9 + 10)_Total (5 + 6 + 9 + 10)',
       'Gross Irrigated Area_Canal_Government',
       'Gross Irrigated Area_Canal_Private',
       'Gross Irrigated Area_Canal_Total (9 To 10)',
       'Gross Irrigated Area_Tank_Tank', 'Gross Irrigated Area_Well_Tubewell',
       'Gross Irrigated Area_Well_Other Well',
       'Gross Irrigated Area_Well_Total (12 To 13)',
       'Gross Irrigated Area_Other Source_Other Source',
       'Gross Irrigated Area_Total (14 + 15 + 18 + 19)_Total (14 + 15 + 18 + 19)'],
      dtype='object')

In [22]:
final_df.to_csv('Source_Irrigated_Area.csv')

In [23]:
df1 = pd.read_csv('Area_under_crops.csv')
df2 = pd.read_csv('Crop_Irrigated_Area.csv')
df3 = pd.read_csv('Source_Irrigated_Area.csv')
df4 = pd.read_csv('Crop Yield final.csv')

In [24]:
import pandas as pd
import numpy as np

# Let's assume your data is loaded into a DataFrame 'df3'


# --- Step 1: Create the 'State' column ---
# We find rows where the 'District' column contains "State:" and use that value.
# Otherwise, the value will be NaN (empty).
df3['State'] = np.where(df3['District'].str.contains('State:', na=False), df3['District'], np.nan)


# --- Step 2: Forward-fill the state names ---
# This is the key step. It propagates the last valid state name downwards.
df3['State'].ffill(inplace=True)


# --- Step 3: Clean up the data ---
# Remove the "State: " prefix from the 'State' column for a clean name
df3['State'] = df3['State'].str.replace('State: ', '', regex=False).str.strip()

# Remove the original rows that were state headers
df3 = df3[~df3['District'].str.contains('State:', na=False)].copy()

# Remove any rows that are entirely empty, if they exist
df3.dropna(how='all', inplace=True)


# --- Final Result ---
print("✅ State names have been preserved for each district.")
display(df3.head())

✅ State names have been preserved for each district.


/var/folders/y7/ch6tzcy15m581gv9c6rl_zjm0000gn/T/ipykernel_14788/4294019913.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df3['State'].ffill(inplace=True)


,Unnamed: 0,Year,State,District,Net Irrigated Area_Canal_Government,Net Irrigated Area_Canal_Private,Net Irrigated Area_Canal_Total (3 To 4),Net Irrigated Area_Tank_Tank,Net Irrigated Area_Well_Tubewell,Net Irrigated Area_Well_Other Well,...,Net Irrigated Area_Total (5 + 6 + 9 + 10)_Total (5 + 6 + 9 + 10),Gross Irrigated Area_Canal_Government,Gross Irrigated Area_Canal_Private,Gross Irrigated Area_Canal_Total (9 To 10),Gross Irrigated Area_Tank_Tank,Gross Irrigated Area_Well_Tubewell,Gross Irrigated Area_Well_Other Well,Gross Irrigated Area_Well_Total (12 To 13),Gross Irrigated Area_Other Source_Other Source,Gross Irrigated Area_Total (14 + 15 + 18 + 19)_Total (14 + 15 + 18 + 19)
0,0,2018,NaN,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
2,2,2018,Andhra Pradesh,Anantapur,14323,-,14323,1755,125330,195,...,141991,16196,-,16196,2016,153088,195,153283,479,171974
3,3,2018,Andhra Pradesh,Chittoor,619,-,619,7496,124134,8261,...,140510,619,-,619,7804,156601,10332,166933,-,175356
4,4,2018,Andhra Pradesh,East godavari,169922,-,169922,16793,68760,32,...,277346,316411,-,316411,21248,115255,36,115291,27024,479974
5,5,2018,Andhra Pradesh,Guntur,308944,-,308944,3514,82515,4352,...,412884,341414,-,341414,3642,135801,5143,140944,15005,501005


In [25]:
df1

,Unnamed: 0,Year,State,District,Cereals & Millets_Rice,Cereals & Millets_Rice.1,Cereals & Millets_Rice.2,Cereals & Millets_Rice.3,Cereals & Millets_Rice.4,Cereals & Millets_Jowar,...,Green Manure_Green Manure,Other Non Food Crops_Other Non Food Crops,Total Non Food Crop_Total Non Food Crop,Total Cropped Area_Total Cropped Area,Area Sown More Than Once_Area Sown More Than Once,Net Area Sown_Net Area Sown,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111
0,0,2009,NaN,District,Kharif,Autumn,Winter,Summer,Total (3 To 6),Kharif,...,NaN,NaN,(79 + 85 + 88 + 97 + 98 + 99 + 100 + 101 + 102),NaN,(104 - 106),NaN,NaN,NaN,NaN,NaN
1,1,2009,NaN,2,3,4,5,6,7,8,...,101.0,102.0,103,104.0,105,106.0,NaN,NaN,NaN,NaN
2,2,2009,NaN,State: Andaman and Nicobar Islands,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2009,NaN,Nicobars,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,230,377.0,109,268.0,NaN,NaN,NaN,NaN
4,4,2009,NaN,North and middle andaman,NaN,4969,NaN,1334,6303,NaN,...,NaN,1.0,461,9009.0,1470,7539.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14053,14053,2010,NaN,Medinipur west,NaN,39370,403460,173888,616718,NaN,...,NaN,3416.0,96562,878493.0,392294,486199.0,NaN,NaN,NaN,NaN
14054,14054,2010,NaN,Murshidabad,NaN,11867,170696,121612,304175,NaN,...,NaN,975.0,273133,866829.0,470871,395958.0,NaN,NaN,NaN,NaN
14055,14055,2010,NaN,Nadia,NaN,48392,88778,98567,235737,NaN,...,NaN,6983.0,225446,672312.0,381865,290447.0,NaN,NaN,NaN,NaN
14056,14056,2010,NaN,Purba bardhaman,NaN,9285,364128,189443,562856,NaN,...,NaN,137.0,55233,747942.0,295480,452462.0,NaN,NaN,NaN,NaN


In [26]:
import pandas as pd

# Let's assume your three DataFrames are named df1, df2, and df3

# --- Step 1: Add custom suffixes to each DataFrame's columns ---

# Identify the columns to rename in each DataFrame (all except the merge keys)
cols1_to_rename = [col for col in df1.columns if col not in ['Year', 'District','State']]
cols2_to_rename = [col for col in df2.columns if col not in ['Year', 'District','State']]
cols3_to_rename = [col for col in df3.columns if col not in ['Year', 'District','State']]

# Create renaming dictionaries
rename_dict1 = {col: f"{col}_Area" for col in cols1_to_rename}
rename_dict2 = {col: f"{col}_irrigated_area" for col in cols2_to_rename}
rename_dict3 = {col: f"{col}_source_irr_area" for col in cols3_to_rename}

# Apply the renaming
df1_renamed = df1.rename(columns=rename_dict1)
df2_renamed = df2.rename(columns=rename_dict2)
df3_renamed = df3.rename(columns=rename_dict3)

print(" Suffixes added to column names.")

# --- Step 2: Merge the renamed DataFrames ---

# Merge the first two
merged_df = pd.merge(df1_renamed, df2_renamed, on=['Year', 'District'], how='outer')

# Merge the result with the third
final_df = pd.merge(merged_df, df3_renamed, on=['Year', 'District'], how='outer')


# --- Step 3: Inspect the final result ---
print("Merge complete!")
print(f"The final DataFrame has {final_df.shape[0]} rows and {final_df.shape[1]} columns.")

# Display the first few rows to see the new column names
print("\nFirst 5 rows of the final merged DataFrame with custom suffixes:")
display(final_df.head())

# Optionally, save your final combined DataFrame
# final_df.to_csv("combined_final_data_with_suffixes.csv", index=False)

✅ Suffixes added to column names.
Merge complete!
The final DataFrame has 14577 rows and 195 columns.

First 5 rows of the final merged DataFrame with custom suffixes:


,Unnamed: 0_Area,Year,State_x,District,Cereals & Millets_Rice_Area,Cereals & Millets_Rice.1_Area,Cereals & Millets_Rice.2_Area,Cereals & Millets_Rice.3_Area,Cereals & Millets_Rice.4_Area,Cereals & Millets_Jowar_Area,...,Net Irrigated Area_Total (5 + 6 + 9 + 10)_Total (5 + 6 + 9 + 10)_source_irr_area,Gross Irrigated Area_Canal_Government_source_irr_area,Gross Irrigated Area_Canal_Private_source_irr_area,Gross Irrigated Area_Canal_Total (9 To 10)_source_irr_area,Gross Irrigated Area_Tank_Tank_source_irr_area,Gross Irrigated Area_Well_Tubewell_source_irr_area,Gross Irrigated Area_Well_Other Well_source_irr_area,Gross Irrigated Area_Well_Total (12 To 13)_source_irr_area,Gross Irrigated Area_Other Source_Other Source_source_irr_area,Gross Irrigated Area_Total (14 + 15 + 18 + 19)_Total (14 + 15 + 18 + 19)_source_irr_area
0,3155.0,1998,NaN,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
1,3727.0,1998,NaN,24 paraganas north,NaN,21417,199701,114251,335369,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3728.0,1998,NaN,24 paraganas south,NaN,4337,363033,80890,448260,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3160.0,1998,NaN,Adilabad,NaN,66373,NaN,12152,78525,49783,...,78528,23762,-,23762,22102,24012,23227,47239,2942,96045
4,3646.0,1998,NaN,Agra,NaN,1536,NaN,NaN,1536,1179,...,225534,40386,-,40386,221,193152,5183,198335,1750,240692


AttributeError: 'Series' object has no attribute 'isnan'

In [27]:
final_df.to_csv('all_merged.csv')

In [28]:
df_new = df4

In [29]:
import pandas as pd
import numpy as np

# Assuming df_new is your loaded DataFrame

# --- Step 1: Clean and Prepare the DataFrame ---
df_new['Year'] = df_new['Year'].astype(str)
df_new['Year'] = df_new['Year'].str.extract(r'(\d{4})')
df_new['Year'] = pd.to_numeric(df_new['Year'], errors='coerce')
df_new.dropna(subset=['Year'], inplace=True)
df_new['Year'] = df_new['Year'].astype(int)

df_new.columns = [col.strip().replace('\n', ' ') for col in df_new.columns]
print(" Cleaned DataFrame Columns:")
print(df_new.columns)
print("-" * 30)


# --- Step 2: Reshape the data using the CORRECT column name ---

# The column name below has been corrected to match the output exactly.
df_wide = df_new.pivot_table(
    index=['Year', 'State', 'District'],
    columns=['Crop Name', 'Crop Season'],
    values='Land Area Utilized For Production (UOM:Ha(Hectare)), Scaling Factor:1',
    aggfunc='sum'
)

# --- Step 3: Flatten the Column Headers ---
if isinstance(df_wide.columns, pd.MultiIndex):
    df_wide.columns = ['_'.join(map(str, col)).strip() for col in df_wide.columns.values]

df_wide.reset_index(inplace=True)

# --- Final Result ---
print("\nReshaping complete!")
display(df_wide.head())

✅ Cleaned DataFrame Columns:
Index(['Country', 'State', 'District', 'Year', 'Crop Name', 'Crop Season',
       'Land Area Utilized For Production (UOM:Ha(Hectare)), Scaling Factor:1',
       'Crop Production (UOM:t(Tonne)), Scaling Factor:1',
       'Crop Yield (UOM:t/Ha(TonnesperHectare)), Scaling Factor:1'],
      dtype='object')
------------------------------

Reshaping complete!


,Year,State,District,Arecanut_Kharif,Arecanut_Rabi,Arecanut_Whole Year,Arhar/Tur_Autumn,Arhar/Tur_Kharif,Arhar/Tur_Rabi,Arhar/Tur_Summer,...,Urad_Winter,Wheat_Kharif,Wheat_Rabi,Wheat_Summer,Wheat_Whole Year,Wheat_Winter,other oilseeds_Kharif,other oilseeds_Rabi,other oilseeds_Summer,other oilseeds_Whole Year
0,1997,Andhra Pradesh,Ananthapuramu,NaN,NaN,NaN,NaN,21400.0,NaN,NaN,...,NaN,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,Andhra Pradesh,Chittoor,NaN,NaN,NaN,NaN,6100.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,Andhra Pradesh,East Godavari,NaN,NaN,NaN,NaN,1600.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,Andhra Pradesh,Guntur,NaN,NaN,NaN,NaN,28400.0,1800.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997,Andhra Pradesh,Krishna,NaN,NaN,NaN,NaN,10700.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
cols1_to_rename = [col for col in df_wide.columns if col not in ['Year', 'District','State']]


# Create renaming dictionaries
rename_dict1 = {col: f"{col}_Yield" for col in cols1_to_rename}


# Apply the renaming
df_wide = df_wide.rename(columns=rename_dict1)


0           Uttarakhand
1                   NaN
2                   NaN
3        Andhra Pradesh
4         Uttar Pradesh
              ...      
14572         Telangana
14573         Karnataka
14574           Haryana
14575        Puducherry
14576          Nagaland
Name: State, Length: 14577, dtype: object

In [50]:
import pandas as pd

# Let's assume your two DataFrames are named final_df and df_wide

print("DataFrames before merging:")
print(f"final_df shape: {final_df.shape}")
print(f"df_wide shape:  {df_wide.shape}")

# Perform the outer merge with the specified suffixes.
fully_merged_df = pd.merge(
    final_df,          # Left DataFrame, suffix is ''
    df_wide,           # Right DataFrame, suffix is '_yield'
    on=['Year', 'District','State'],
    how='outer',
    suffixes=('', '_yield') # This correctly applies the suffixes
)

# Inspect the final result
print("\n Merge complete!")
print(f"The fully merged DataFrame has {fully_merged_df.shape[0]} rows and {fully_merged_df.shape[1]} columns.")

print("\nFirst 5 rows of the fully merged DataFrame:")
display(fully_merged_df.head())

# Optionally, save your final combined DataFrame
# fully_merged_df.to_csv("fully_merged_dataset.csv", index=False)

DataFrames before merging:
final_df shape: (14577, 195)
df_wide shape:  (13767, 241)

✅ Merge complete!
The fully merged DataFrame has 19153 rows and 433 columns.

First 5 rows of the fully merged DataFrame:


,Unnamed: 0_Area,Year,State_x,District,Cereals & Millets_Rice_Area,Cereals & Millets_Rice.1_Area,Cereals & Millets_Rice.2_Area,Cereals & Millets_Rice.3_Area,Cereals & Millets_Rice.4_Area,Cereals & Millets_Jowar_Area,...,Urad_Winter_Yield,Wheat_Kharif_Yield,Wheat_Rabi_Yield,Wheat_Summer_Yield,Wheat_Whole Year_Yield,Wheat_Winter_Yield,other oilseeds_Kharif_Yield,other oilseeds_Rabi_Yield,other oilseeds_Summer_Yield,other oilseeds_Whole Year_Yield
0,NaN,1997,NaN,Adilabad,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1997,NaN,Agra,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,123600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1997,NaN,Ahmadabad,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,96000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1997,NaN,Ahmednagar,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,79700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1997,NaN,Aizawl,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
fully_merged_df.State

0            Telangana
1        Uttar Pradesh
2              Gujarat
3          Maharashtra
4              Mizoram
             ...      
19148        Telangana
19149        Karnataka
19150          Haryana
19151       Puducherry
19152         Nagaland
Name: State, Length: 19153, dtype: object

In [54]:
fully_merged_df.to_csv('All_in_one.csv')

In [43]:
fully_merged_df.District

0           Adilabad
1               Agra
2          Ahmadabad
3         Ahmednagar
4             Aizawl
            ...     
18821        Yadadri
18822         Yadgir
18823    Yamunanagar
18824          Yanam
18825      Zunheboto
Name: District, Length: 18826, dtype: object

In [35]:
df3

,Unnamed: 0,Year,State,District,Net Irrigated Area_Canal_Government,Net Irrigated Area_Canal_Private,Net Irrigated Area_Canal_Total (3 To 4),Net Irrigated Area_Tank_Tank,Net Irrigated Area_Well_Tubewell,Net Irrigated Area_Well_Other Well,...,Net Irrigated Area_Total (5 + 6 + 9 + 10)_Total (5 + 6 + 9 + 10),Gross Irrigated Area_Canal_Government,Gross Irrigated Area_Canal_Private,Gross Irrigated Area_Canal_Total (9 To 10),Gross Irrigated Area_Tank_Tank,Gross Irrigated Area_Well_Tubewell,Gross Irrigated Area_Well_Other Well,Gross Irrigated Area_Well_Total (12 To 13),Gross Irrigated Area_Other Source_Other Source,Gross Irrigated Area_Total (14 + 15 + 18 + 19)_Total (14 + 15 + 18 + 19)
0,0,2018,NaN,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
2,2,2018,Andhra Pradesh,Anantapur,14323,-,14323,1755,125330,195,...,141991,16196,-,16196,2016,153088,195,153283,479,171974
3,3,2018,Andhra Pradesh,Chittoor,619,-,619,7496,124134,8261,...,140510,619,-,619,7804,156601,10332,166933,-,175356
4,4,2018,Andhra Pradesh,East godavari,169922,-,169922,16793,68760,32,...,277346,316411,-,316411,21248,115255,36,115291,27024,479974
5,5,2018,Andhra Pradesh,Guntur,308944,-,308944,3514,82515,4352,...,412884,341414,-,341414,3642,135801,5143,140944,15005,501005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12930,12930,2002,Uttarakhand,Pithoragarh,1183,411,1594,-,-,-,...,4073,2155,584,2739,-,-,-,-,4970,7709
12931,12931,2002,Uttarakhand,Rudra prayag,1079,147,1226,-,-,-,...,2363,2062,270,2332,-,-,-,-,2189,4521
12932,12932,2002,Uttarakhand,Tehri garhwal,682,1324,2006,-,-,-,...,7913,1268,2532,3800,-,-,-,-,11258,15058
12933,12933,2002,Uttarakhand,Udam singh nagar,26182,-,26182,42,99205,12338,...,145226,56003,-,56003,1011,118922,58996,177918,1683,236615
